In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# For EfficientNet
efficientNetTransform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomCrop(224, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(2.8),
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Test/Validation data transformations (same normalization)
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
])

# Load datasets
# use download=True to download the dataset if not already present
train_set = torchvision.datasets.CIFAR100(root='./data', train=True, download=False, transform=efficientNetTransform_train)
test_set = torchvision.datasets.CIFAR100(root='./data', train=False, download=False, transform=transform_test)

# Train/validation split
train_size = int(0.9 * len(train_set))
val_size = len(train_set) - train_size
train_set, val_set = torch.utils.data.random_split(train_set, [train_size, val_size])

# Print dataset sizes
print("Train samples:", len(train_set))
print("Validation samples:", len(val_set))
print("Test samples:", len(test_set))

# DataLoaders
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=64, shuffle=False)




In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
import matplotlib.pyplot as plt

# Check for MPS (Metal Performance Shaders) support for MacOs
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

# Define Functions
def initialize_model(model_name='ResNet'):
    if model_name == 'ResNet':
        model = models.resnet18(weights=None)
        model.fc = nn.Linear(model.fc.in_features, 100)
    elif model_name == 'DenseNet':
        model = models.densenet121(weights=None)
        model.classifier = nn.Linear(model.classifier.in_features, 100)
    elif model_name == 'EfficientNet':
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 100)
    else:
        raise ValueError("Model not supported")
    
    return model

def evaluate(model, dataloader):
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in dataloader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return 100 * correct / total

def train_model(model, train_loader, val_loader, device, num_epochs=10, lr=0.001):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_accuracies = []
    val_accuracies = []

    for epoch in range(num_epochs):
        model.train()
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            if i % 100 == 0:
                print(f"[Epoch {epoch+1}] Batch {i}/{len(train_loader)} - Loss: {loss.item():.4f}")

        train_accuracy = 100 * correct / total
        train_accuracies.append(train_accuracy)

        val_accuracy = evaluate(model, val_loader)
        val_accuracies.append(val_accuracy)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%")

    return train_accuracies, val_accuracies

def plot_accuracy(train_accuracies, val_accuracies):
    plt.plot(range(1, len(train_accuracies)+1), train_accuracies, label='Train Accuracy')
    plt.plot(range(1, len(val_accuracies)+1), val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy - ResNet')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Base Training Testing
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = initialize_model("ResNet")
train_accuracies, val_accuracies = train_model(model, train_loader, val_loader, device)
plot_accuracy(train_accuracies, val_accuracies)

final_test_accuracy = evaluate(model, test_loader)
print(f"Test Accuracy: {final_test_accuracy:.2f}%")

In [ ]:
# Hyperparameter tuning
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
optimizers = ['adam', 'sgd', 'rmsprop']
learning_rate = 0.001
batch_size = 32

for opt_name in optimizers:
    print(f"Running with Optimizer={opt_name}, LR={learning_rate}, Batch Size={batch_size}")

    # Recreate DataLoaders with fixed batch size
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_set, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = initialize_model("ResNet").to(device)

    # Set optimizer
    if opt_name == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    elif opt_name == 'sgd':
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
    else:
        optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

    # Train model
    train_accuracies, val_accuracies = train_model(
        model, train_loader, val_loader, device, 50, learning_rate
    )

    # Plot (optional per run)
    plot_accuracy(train_accuracies, val_accuracies)

    final_test_accuracy = evaluate(model, test_loader)
    print(f"Test Accuracy: {final_test_accuracy:.2f}%")